In [1]:
import json
import requests

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display

In [3]:
def submit_to_ars(m):
    submit_url = 'https://ars.transltr.io/ars/api/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'https://arax.ncats.io/?source=ARS&id={message_id}')
    return message_id

In [4]:
def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    for child in j['children']:
        if child['status'] == 'Done':
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            try:
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except:
                nresults=0
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return results

In [5]:
def printjson(j):
    print(json.dumps(j,indent=4))

In [6]:
def name_lookup(text):
    url= f'https://name-resolution-sri.renci.org/lookup?string={text}&offset=0&limit=10'
    response = requests.post(url)
    printjson(response.json())

In [7]:
def one_hop_message(subject_category,object_category,predicate=None,subject_id = None, object_id = None):
    query_graph = {
    "nodes": {
        'a':{
            "category": subject_category
        },
        'b':{
            "category": object_category
        }
    },
    "edges": {
        'ab':{
            "subject": "a",
            "object": "b",
            #"predicate": predicate
        }
    }
    }
    if predicate is not None:
        query_graph['edges']['ab']['predicate'] = predicate
    if subject_id is not None:
        query_graph['nodes']['a']['id'] = subject_id
    if object_id is not None:
        query_graph['nodes']['b']['id'] = object_id
    message = {"message": {"query_graph": query_graph},'knowledge_graph':{"nodes": [], "edges": [],}, 'results':[]}
    return message

In [9]:
name_lookup('glycerol kinase')

{
    "www:bget?hsa:2710": [
        "GKD; glycerol kinase",
        "K00864 glycerol kinase [EC:2.7.1.30] | (RefSeq) GK",
        "GK1"
    ],
    "www:bget?hsa:2712": [
        "GKTA; glycerol kinase 2",
        "K00864 glycerol kinase [EC:2.7.1.30] | (RefSeq) GK2",
        "GKP2"
    ],
    "www:bget?hsa:55750": [
        "MULK; acylglycerol kinase",
        "K09881 acylglycerol kinase [EC:2.7.1.94] | (RefSeq) AGK",
        "CATC5",
        "CTRCT38",
        "MTDPS10"
    ],
    "MONDO:0010613": [
        "glycerol kinase deficiency",
        "glycerol kinase deficiency; GKD",
        "inborn error of glycerol kinase activity",
        "inborn glycerol kinase activity disorder",
        "rare inborn error of glycerol kinase activity",
        "rare inborn error of glycerol kinase activity",
        "GKD",
        "GK deficiency",
        "GK1 deficiency",
        "hyperglycerolemia"
    ],
    "www:bget?ko:K00864": [
        "glycerol kinase [EC:2.7.1.30]",
        "GK",
        "g

Chemicals related to GK (UniProtKB:P32189)

In [8]:
m = one_hop_message('biolink:ChemicalSubstance','biolink:Gene',object_id='UniProtKB:P32189')

In [9]:
printjson(m)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:ChemicalSubstance"
                },
                "b": {
                    "category": "biolink:Gene",
                    "id": "UniProtKB:P32189"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}


In [10]:
mid = submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=b64f6695-3c5d-479f-8d8f-138337d63ff3


In [12]:
j = retrieve_ars_results(mid)

Done
Done ara-aragorn 38
Done ara-arax 56
Done ara-bte 85
Done ara-unsecret 0
Done kp-genetics 0
Error kp-molecular 0
Done ara-explanatory 0
Done ara-improving 0
Done kp-cam 0
Done kp-textmining 0
Unknown ara-ncats 0
Unknown ara-robokop 0
Done ara-aragorn-exp 39
Error kp-openpredict 0
Done kp-icees 0


In [12]:
j['ara-aragorn']

{'message': {'query_graph': {'nodes': {'b': {'id': ['UniProtKB:P32189'],
     'category': ['biolink:Gene'],
     'is_set': False},
    'a': {'id': None,
     'category': ['biolink:ChemicalSubstance'],
     'is_set': False}},
   'edges': {'ab': {'subject': 'a',
     'object': 'b',
     'predicate': None,
     'relation': None}}},
  'knowledge_graph': {'nodes': {'CHEBI:72329': {'category': 'biolink:ChemicalSubstance',
     'name': 'HYDROCORTISONE PHOSPHORIC ACID',
     'attributes': [{'type': 'biolink:id',
       'value': ['CHEMBL.COMPOUND:CHEMBL1641',
        'CHEMBL.COMPOUND:CHEMBL1201526',
        'CHEMBL.COMPOUND:CHEMBL1200340',
        'CHEMBL.COMPOUND:CHEMBL1230695',
        'CHEMBL.COMPOUND:CHEMBL144874',
        'CHEMBL.COMPOUND:CHEMBL1200968',
        'DRUGBANK:DB00741',
        'DRUGBANK:DB00198',
        'DRUGBANK:DB14542',
        'PUBCHEM.COMPOUND:136254572',
        'PUBCHEM.COMPOUND:441407',
        'PUBCHEM.COMPOUND:440214',
        'PUBCHEM.COMPOUND:78000',
        'PUBC

In [14]:
view = GammaViewer(props={"data":j['ara-aragorn-exp']})
display(view)

What are the relationships between GK and glycerol (also called glycerin) ?

What are the id's for glycerol?

In [41]:
resp = requests.get('https://nodenormalization-sri.renci.org/get_normalized_nodes?curie=CHEBI:17754')
js = resp.json()
glycerol_ids = set([x['identifier'] for x in js['CHEBI:17754']['equivalent_identifiers']])
print(glycerol_ids)

{'CHEBI:17754', 'MESH:C043941', 'MESH:D005990', 'INCHIKEY:FCMSAPXDFYCBGJ-PWJLMRLQSA-N', 'GTOPDB:5195', 'DRUGBANK:DB09462', 'HMDB:HMDB0000131', 'PUBCHEM.COMPOUND:753', 'MESH:D003451', 'UNII:PDC6A3C0OX', 'KEGG.COMPOUND:C00116', 'CHEMBL.COMPOUND:CHEMBL692'}


In [30]:
m = one_hop_message('biolink:ChemicalSubstance','biolink:Gene',object_id='NCBIGENE:2710')

In [31]:
printjson(m)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:ChemicalSubstance"
                },
                "b": {
                    "category": "biolink:Gene",
                    "id": "NCBIGENE:2710"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}


In [32]:
mid = submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=d64b7e98-e058-4447-98f7-d62fa18afde6


In [44]:
jres = retrieve_ars_results(mid)

Done
Done ara-aragorn 38
Done ara-arax 56
Done ara-bte 85
Done ara-unsecret 0
Done kp-genetics 0
Error kp-molecular 0
Done ara-explanatory 0
Done ara-improving 0
Done kp-cam 0
Done kp-textmining 0
Unknown ara-ncats 0
Unknown ara-robokop 0
Done ara-aragorn-exp 39
Error kp-openpredict 0
Done kp-icees 0


In [46]:
for responder in jres:
    if len(jres[responder]['message']['results']) > 0:
        kg = jres[responder]['message']['knowledge_graph']
        for edge_id, edge in kg['edges'].items():
            if edge['subject'] in glycerol_ids:
                print(responder, 'Glycerin', edge['predicate'], 'GK')
            if edge['object'] in glycerol_ids:
                print(responder, 'GK', edge['predicate'], 'Glycerin')

ara-aragorn Glycerin biolink:physically_interacts_with GK
ara-arax Glycerin biolink:coexists_with GK
ara-arax Glycerin biolink:part_of GK
ara-arax Glycerin biolink:physically_interacts_with GK
ara-arax Glycerin biolink:positively_regulates_entity_to_entity GK
ara-arax Glycerin biolink:related_to GK
ara-arax GK biolink:coexists_with Glycerin
ara-arax GK biolink:negatively_regulates_entity_to_entity Glycerin
ara-arax GK biolink:physically_interacts_with Glycerin
ara-arax GK biolink:positively_regulates_entity_to_entity Glycerin
ara-arax GK biolink:has_normalized_google_distance_with Glycerin
ara-bte Glycerin biolink:entity_negatively_regulates_entity GK
ara-bte Glycerin biolink:entity_negatively_regulated_by_entity GK
ara-bte Glycerin biolink:physically_interacts_with GK
ara-bte Glycerin biolink:entity_positively_regulates_entity GK
ara-bte Glycerin biolink:entity_positively_regulated_by_entity GK
ara-aragorn-exp Glycerin biolink:physically_interacts_with GK
ara-aragorn-exp GK biolink:ha

In [21]:
view = GammaViewer(props={"data":j['ara-aragorn']})
display(view)

In [15]:
m = one_hop_message('biolink:ChemicalSubstance','biolink:Gene',object_id='NCBIGene:2710')

In [16]:
response = requests.post('https://automat.renci.org/kegg/query',json=m)

In [17]:
response.json()

{'message': {'query_graph': {'nodes': {'a': {'id': None,
     'category': 'biolink:ChemicalSubstance',
     'is_set': False},
    'b': {'id': 'NCBIGene:2710', 'category': 'biolink:Gene', 'is_set': False}},
   'edges': {'ab': {'subject': 'a',
     'object': 'b',
     'predicate': None,
     'relation': None}}},
  'knowledge_graph': {'nodes': {'CHEBI:15422': {'category': ['biolink:NamedThing',
      'biolink:MolecularEntity',
      'biolink:Entity',
      'biolink:ChemicalSubstance',
      'biolink:BiologicalEntity'],
     'name': 'ATP',
     'attributes': [{'type': 'NA',
       'value': ["adenosine 5'-(tetrahydrogen triphosphate)",
        'ATP',
        "Adenosine 5'-triphosphate",
        'Adenosine triphosphate',
        "ADENOSINE-5'-TRIPHOSPHATE",
        'H4atp'],
       'name': 'synonyms',
       'url': None,
       'source': None},
      {'type': 'NA',
       'value': 'ATP',
       'name': 'name',
       'url': None,
       'source': None},
      {'type': 'NA',
       'value': [

In [18]:
view = GammaViewer(props={"data":response.json()})
display(view)

What's the relationship to the disease GKD?

In [48]:
m = one_hop_message('biolink:ChemicalSubstance','biolink:Disease',object_id='MONDO:0010613')
mdc_id =  submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=9644617f-4e5d-4ae7-8059-ab73b4a75cc6


In [51]:
j_mdc = retrieve_ars_results(mdc_id)

Done
Done ara-aragorn 327
Done ara-arax 23
Done ara-bte 326
Done ara-unsecret 112
Done kp-genetics 0
Error kp-molecular 0
Done ara-explanatory 0
Done ara-improving 0
Error kp-cam 0
Done kp-textmining 0
Running ara-ncats 0
Running ara-robokop 0


In [63]:
for responder in j_mdc:
    if len(j_mdc[responder]['message']['results']) > 0:
        kg = j_mdc[responder]['message']['knowledge_graph']
        for edge_id, edge in kg['edges'].items():
            if edge['subject'] in glycerol_ids:
                print(responder, 'Glycerin', edge['predicate'], 'GKD')
            if edge['object'] in glycerol_ids:
                print(responder, 'GKD', edge['predicate'], 'Glycerin')

ara-aragorn Glycerin biolink:related_to GKD
ara-aragorn Glycerin biolink:related_to GKD
ara-arax Glycerin biolink:participates_in GKD
ara-arax Glycerin biolink:related_to GKD
ara-arax Glycerin biolink:treats GKD
ara-arax GKD biolink:has_normalized_google_distance_with Glycerin
ara-bte Glycerin biolink:related_to GKD
ara-unsecret Glycerin biolink:related_to GKD


Suppose that we knew we wanted to consume glycerol.  What genes might help?  What relationships do we need to look for?

In [56]:
m = one_hop_message('biolink:ChemicalSubstance','biolink:Gene',subject_id='CHEBI:17754')
printjson(m)
mdc_id =  submit_to_ars(m)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:ChemicalSubstance",
                    "id": "CHEBI:17754"
                },
                "b": {
                    "category": "biolink:Gene"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}
https://arax.ncats.io/?source=ARS&id=70f4ee19-673a-4578-a692-3f5f6111728d


In [53]:
genesconsume_id = '70f4ee19-673a-4578-a692-3f5f6111728d'

In [54]:
j_genes_consume = retrieve_ars_results(genesconsume_id)

Done
Done ara-aragorn 699
Done ara-arax 100
Done ara-bte 1230
Done ara-unsecret 2569
Done kp-genetics 0
Done kp-molecular 70
Done ara-explanatory 0
Done ara-improving 0
Error kp-cam 0
Done kp-textmining 111
Unknown ara-ncats 0
Unknown ara-robokop 0
Done ara-aragorn-exp 98


In [59]:
from collections import defaultdict
for responder in j_genes_consume:
    if len(j_genes_consume[responder]['message']['results']) > 0:
        kg = j_genes_consume[responder]['message']['knowledge_graph']
        gene2chem = defaultdict(int)
        chem2gene = defaultdict(int)
        for edge_id, edge in kg['edges'].items():
            if edge['subject'] in glycerol_ids:
                try:
                    chem2gene[edge['predicate']]+=1
                except KeyError:
                    chem2gene['??'] += 1
            if edge['object'] in glycerol_ids:
                gene2chem[edge['predicate']]+=1
        for p in chem2gene:
            print(responder, 'Glycerin', p, 'Gene',  chem2gene[p])
        for p in gene2chem:
            print(responder, 'Gene', p, 'Glycerin',  gene2chem[p])

ara-aragorn Glycerin biolink:coexists_with Gene 350
ara-aragorn Glycerin biolink:physically_interacts_with Gene 206
ara-aragorn Glycerin biolink:interacts_with Gene 83
ara-aragorn Glycerin biolink:produced_by Gene 32
ara-aragorn Glycerin biolink:affects Gene 16
ara-aragorn Glycerin biolink:decreases_activity_of Gene 2
ara-aragorn Glycerin biolink:related_to Gene 6
ara-aragorn Glycerin biolink:increases_activity_of Gene 3
ara-aragorn Glycerin biolink:increases_secretion_of Gene 1
ara-arax Glycerin biolink:affects Gene 1
ara-arax Glycerin biolink:coexists_with Gene 11
ara-arax Glycerin biolink:has_input Gene 1
ara-arax Glycerin biolink:increases_activity_of Gene 2
ara-arax Glycerin biolink:negatively_regulates_entity_to_entity Gene 12
ara-arax Glycerin biolink:part_of Gene 11
ara-arax Glycerin biolink:physically_interacts_with Gene 49
ara-arax Glycerin biolink:positively_regulates_entity_to_entity Gene 12
ara-arax Glycerin biolink:related_to Gene 47
ara-arax Glycerin biolink:has_normaliz

In [62]:
j_genes_consume['ara-aragorn-exp']

{'message': {'query_graph': {'nodes': {'b': {'category': ['biolink:Gene'],
     'is_set': True},
    'a': {'id': ['CHEBI:17754'],
     'category': ['biolink:ChemicalSubstance'],
     'is_set': False},
    'extra_qn_0': {'category': ['biolink:NamedThing',
      'biolink:BiologicalEntity',
      'biolink:ChemicalSubstance',
      'biolink:MolecularEntity'],
     'is_set': False}},
   'edges': {'ab': {'subject': 'a', 'object': 'b'},
    'extra_qe_0': {'subject': 'extra_qn_0', 'object': 'b'}}},
  'knowledge_graph': {'nodes': {'UMLS:C0033551': {'category': 'biolink:Gene',
     'name': 'UMLS:C0033551',
     'attributes': [{'type': 'omnicorp_article_count', 'value': 0}]},
    'NCBIGENE:253738': {'category': 'biolink:Gene',
     'name': 'EBF3',
     'attributes': [{'type': 'omnicorp_article_count', 'value': 0}]},
    'UMLS:C0072399': {'category': 'biolink:Gene',
     'name': 'UMLS:C0072399',
     'attributes': [{'type': 'omnicorp_article_count', 'value': 0}]},
    'HGNC:4298': {'category': 'bi

In [59]:
m['message']['query_graph']['edges']['ab']['predicate']="biolink:negatively_regulates_entity_to_entity"

In [61]:
printjson(m)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:ChemicalSubstance",
                    "id": "CHEBI:17754"
                },
                "b": {
                    "category": "biolink:Gene"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:negatively_regulates_entity_to_entity"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}


In [62]:
mdc_id =  submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=89a92acc-c05f-44f1-b14c-bb45554a90f8


In [63]:
m['message']['query_graph']['edges']['ab']['subject']="b"
m['message']['query_graph']['edges']['ab']['object']="a"
printjson(m)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:ChemicalSubstance",
                    "id": "CHEBI:17754"
                },
                "b": {
                    "category": "biolink:Gene"
                }
            },
            "edges": {
                "ab": {
                    "subject": "b",
                    "object": "a",
                    "predicate": "biolink:negatively_regulates_entity_to_entity"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}


In [64]:
mdc_id =  submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=36b6092f-6b8b-4cea-bcb5-8ffb8d51c33c


In [65]:
m = one_hop_message('biolink:ChemicalSubstance','biolink:ChemicalSubstance',subject_id='CHEBI:17754')
printjson(m)

{
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "category": "biolink:ChemicalSubstance",
                    "id": "CHEBI:17754"
                },
                "b": {
                    "category": "biolink:ChemicalSubstance"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b"
                }
            }
        }
    },
    "knowledge_graph": {
        "nodes": [],
        "edges": []
    },
    "results": []
}


In [66]:
gid = submit_to_ars(m)

https://arax.ncats.io/?source=ARS&id=feebe512-947a-43f9-b9d8-5e558cbd26ac


In [64]:
j_chem = retrieve_ars_results('feebe512-947a-43f9-b9d8-5e558cbd26ac')

Done
Done ara-aragorn 8965
Done ara-arax 100
Done ara-bte 8252
Done ara-unsecret 3374
Done kp-genetics 0
Done kp-molecular 2
Done ara-explanatory 0
Done ara-improving 0
Error kp-cam 0
Done kp-textmining 0
Unknown ara-ncats 0
Unknown ara-robokop 0
Done ara-aragorn-exp 0


In [66]:
for responder in j_chem:
    if len(j_chem[responder]['message']['results']) > 0:
        kg = j_chem[responder]['message']['knowledge_graph']
        gene2chem = defaultdict(int)
        chem2gene = defaultdict(int)
        for edge_id, edge in kg['edges'].items():
            if edge['subject'] in glycerol_ids:
                try:
                    chem2gene[edge['predicate']]+=1
                except KeyError:
                    chem2gene['??'] += 1
            if edge['object'] in glycerol_ids:
                try:
                    gene2chem[edge['predicate']]+=1
                except KeyError:
                    gene2chem['??'] += 1 
        for p in chem2gene:
            print(responder, 'Glycerin', p, 'Chemical',  chem2gene[p])
        for p in gene2chem:
            print(responder, 'Chemical', p, 'Glycerin',  gene2chem[p])

ara-aragorn Glycerin biolink:related_to Chemical 7967
ara-aragorn Glycerin biolink:coexists_with Chemical 675
ara-aragorn Glycerin biolink:physically_interacts_with Chemical 313
ara-aragorn Glycerin biolink:correlated_with Chemical 1
ara-aragorn Glycerin biolink:derives_into Chemical 6
ara-aragorn Glycerin biolink:affects Chemical 1
ara-aragorn Chemical biolink:has_part Glycerin 1
ara-aragorn Chemical biolink:correlated_with Glycerin 1
ara-aragorn Chemical biolink:coexists_with Glycerin 1
ara-aragorn Chemical biolink:derives_into Glycerin 1
ara-aragorn Chemical biolink:physically_interacts_with Glycerin 1
ara-arax Glycerin biolink:close_match Chemical 2
ara-arax Glycerin biolink:coexists_with Chemical 13
ara-arax Glycerin biolink:derives_into Chemical 5
ara-arax Glycerin biolink:has_attribute Chemical 1
ara-arax Glycerin biolink:located_in Chemical 1
ara-arax Glycerin biolink:negatively_regulates_entity_to_entity Chemical 8
ara-arax Glycerin biolink:part_of Chemical 7
ara-arax Glycerin